In [1]:
%load_ext autoreload
%autoreload 2
    

In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
import bets
import os
import glob
import datetime as dt
import time
#import ot
import json
import requests
import statsmodels.api as sm 
from tqdm import tqdm
pd.set_option('display.float_format', lambda x: '%.3f' % x)

nba = bets.nba()
import seaborn as sns
import matplotlib.pyplot as plt
import re
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
from nba_api.stats.endpoints import leaguegamefinder,BoxScoreSummaryV2,LeagueDashOppPtShot,PlayerProfileV2,GameRotation,LeagueDashPlayerShotLocations

What changes to make to this model
1. Want to have decay for our rolling estimates  
   a.This will be both at the team level (these are currently in the query) and player level  
   b.this will have to be done in pandas, in the script  
   c. need to re-write sql query to pull in values not adjusted  
2. New One-Hot Variables  
   a. Top Scorer -  highest moving average on team  
   b. Top Shooter - Takes most threes on team  
   c. Volume Shooter - in  90th+ percentile of total threes taken  
   d. corner merchant - over 33% of threes from corner - look into number  
   e. starter - this is created but can help to be added
   f. nans filled
   g. reduce threes made to a max of 8 and bin if they made more than 8 in a game
4. Interaction Terms:  
   a. Rolling Three % * Home  
   b. Player Off rating - team off rating  
   c.
5. Add-in odds

In [95]:
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss
from sklearn.preprocessing import OneHotEncoder

In [3]:
def probOutput(preds,y):
    c = [12,11,10,9,8,7,6,5,4,3,2,1,0]
    finalo = pd.DataFrame(np.array([v for r in preds[c].cumsum(axis=1).values for v in r]).reshape(preds[c].shape),
                         columns=c,index=preds.index)
    
    df = finalo.filter(preds.columns).join(y)
    return df


In [3]:
addMrkts = 'player_points,player_points_alterante,player_rebounds,player_rebounds_alternate,player_first_basket,player_assists,player_assists_alternate'
mrkts = '&markets=player_threes,player_threes_alternate'
apiKey = 'apiKey=153deeb03ca7b659e72d18e28219d1a8'
oddsType = '&oddsFormat=american'
regions = '&regions=us'
books = '&bookmakers=draftkings%2Cfanduel%2Cespnbet'
inputs = ['{}',apiKey,regions,mrkts,oddsType,books]
url = '''https://api.the-odds-api.com/v4/sports/basketball_nba/events/{}/odds?{}{}{}&dateFormat=iso{}{}'''.format(*inputs)

In [5]:
#Going to run the back testing on '24 season

In [98]:
#update model to use '22-'23
trainData = nba.threeData('data/sql/thrTesting.sql')

In [99]:
ogMod = pd.read_pickle('data/model/thrLogit.pkl')
params = ogMod.params.index

In [100]:
X = trainData[trainData.season < '2023-24']
y = trainData[trainData.season < '2023-24'].threesMade.values
X = nba.cleanNaThr(X)
X = nba.scaleData(X.filter(params))

In [101]:
newMod = sm.MNLogit(endog = y, exog = X.filter(params))
upModel = newMod.fit(method = 'ncg',maxiter=100)

Optimization terminated successfully.
         Current function value: 1.726821
         Iterations: 22
         Function evaluations: 26
         Gradient evaluations: 26
         Hessian evaluations: 22


In [102]:
xtest = trainData[trainData.season=='2023-24']
xtest = nba.cleanNaThr(xtest)
xtest = nba.scaleData(xtest.filter(params))

In [103]:
ytest = trainData[trainData.season=='2023-24'].threesMade

In [144]:
ncg23 = upModel.predict(xtest)
og23 = ogMod.predict(xtest)

In [145]:
upOut23 = probOutput(ncg23,ytest)

In [136]:
ogOut23 = probOutput(og23,ytest)

In [148]:
#ncg23[12] = ncg23[[12,13,14]].sum(axis=1)
ncg23 = ncg23.iloc[:,:13]

In [139]:
actualsOvers = [[1] * (y+1) + [0] * (12-y) for y in ytest]

In [140]:
ohe = [[0] * y +[1] + [0]*(12-y) for y in ytest]

array([1.92803307e-01, 2.69227652e-01, 2.30650782e-01, 1.45311614e-01,
       8.47718616e-02, 4.29532377e-02, 2.00178296e-02, 9.07691061e-03,
       2.99862225e-03, 1.45878920e-03, 4.86263068e-04, 1.62087689e-04,
       8.10438447e-05])

In [158]:
def brier_scores(actuals,preds):
    '''Will create a brier score to compare models
    Inputs: an array of actuals one hot encoded, a data frame of predictions from model
    Outputs: score as a float
    '''
    return (preds.sub(actuals) ** 2).sum(axis=1).mean()
def score_comp(actuals,preds,

base = np.tile(np.array(ohe).mean(axis=0), (np.array(ohe).shape[0], 1))

In [161]:
((upOut23.iloc[:,:-1].sub(np.array(actualsOvers))) ** 2).sum(axis=1).mean()

0.8373085269161717

In [164]:
((pd.DataFrame(ohe).sub(np.array(base))) ** 2).sum(axis=1).mean()

0.8065023659200062

In [145]:
import scipy

In [113]:
((ncg23.iloc[:,:-2].sub(actuals) ** 2).sum(axis=1).mean()

AttributeError: 'list' object has no attribute 'toarray'

In [194]:
((cg23.iloc[:,:-2].sub(actuals.toarray())) ** 2).sum(axis=1).mean()

0.7965169469293196

In [176]:
((og23.iloc[:,:-1].sub(actuals.toarray())) ** 2).sum(axis=1).mean()

0.7985511754618587

In [178]:
((pred23.iloc[:,:-2].sub(actuals.toarray())) ** 2).sum(axis=1).mean()

0.7946153987955363

In [27]:
data =  nba.threeData() #there will need to be an edit to only get data for today
data = data[data.season=='2023-24']
model = pd.read_pickle('data/model/thrLogit.pkl')
#create model data and scale
X = nba.cleanNaThr(data)
X = nba.scaleData(X.filter(model.params.index))
#make day's predictions
preds = model.predict(X)

#unders = data[data.game_date==date][['name','team','game_id']].join(finalu.filter(preds.columns))

In [35]:
preds['game_date'] = data['game_date'].values
preds['threesMade'] = data['threesMade'].values

In [36]:
preds

0     1     2     3     4     5     6     7     8     9    10    11  \
0     0.257 0.297 0.195 0.118 0.064 0.032 0.022 0.011 0.003 0.001 0.000 0.000   
1     0.103 0.206 0.244 0.233 0.127 0.049 0.029 0.007 0.001 0.000 0.000 0.000   
2     0.137 0.248 0.231 0.188 0.110 0.050 0.024 0.011 0.001 0.001 0.000 0.000   
3     0.119 0.200 0.228 0.212 0.134 0.057 0.034 0.012 0.003 0.001 0.000 0.000   
4     0.209 0.279 0.248 0.166 0.070 0.017 0.009 0.002 0.000 0.000 0.000 0.000   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
12334 0.230 0.294 0.208 0.145 0.071 0.035 0.010 0.004 0.001 0.000 0.000 0.000   
12335 0.249 0.272 0.160 0.178 0.077 0.047 0.010 0.004 0.001 0.000 0.000 0.000   
12336 0.072 0.178 0.229 0.241 0.143 0.081 0.037 0.013 0.005 0.000 0.000 0.000   
12337 0.299 0.304 0.197 0.112 0.048 0.025 0.009 0.004 0.002 0.000 0.000 0.000   
12338 0.209 0.301 0.243 0.148 0.056 0.027 0.011 0.005 0.001 0.000 0.000 0.000   

         12    13               name   game_date  threesMade  
0     0.000 0.000         Chris Paul  2023-10-24           0  
1     0.000 0.000   D'Angelo Russell  2023-10-24           2  
2     0.000 0.000       Devin Booker  2023-10-24           3  
3     0.000 0.000       Jamal Murray  2023-10-24           3  
4     0.000 0.000     Taurean Prince  2023-10-24           4  
...     ...   ...                ...         ...         ...  
12334 0.000 0.000     Andrew Wiggins  2024-04-14           2  
12335 0.000 0.000        Dario Saric  2024-04-14           1  
12336 0.000 0.000  Bogdan Bogdanovic  2024-04-14           3  
12337 0.000 0.000     Vasilije Micic  2024-04-14           1  
12338 0.000 0.000       LeBron James  2024-04-14           0  

[12339 rows x 17 columns]

In [11]:
df = pd.read_sql('''select season,player_id,name,pts,pl.game_date
                FROM plyrLogs pl JOIN teamLog tl USING (team_id,game_id)
                JOIN players p USING(player_id)
                
                where pl.game_date <'2024-10-22'
                ''',nba.conn)

In [18]:
df.groupby(['season','player_id','name']).pts.mean().reset_index().groupby('season').pts.

In [ ]:
##check model 